## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Skjervoy,NO,70.0311,20.9714,22.98,62,1,13.06,clear sky
1,1,Lagoa,PT,39.0500,-27.9833,63.39,82,63,9.64,broken clouds
2,2,Hilo,US,19.7297,-155.0900,69.24,93,90,0.00,light rain
3,3,Puerto Escondido,MX,15.8500,-97.0667,86.83,62,1,0.00,clear sky
4,4,Ribeira Grande,PT,38.5167,-28.7000,64.24,75,91,11.92,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?65
What is the maximum temperature you would like for your trip?82


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp)&(city_data_df["Max Temp"]<=max_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hilo,US,19.7297,-155.0900,69.24,93,90,0.00,light rain
7,7,Bathsheba,BB,13.2167,-59.5167,79.41,78,40,14.97,scattered clouds
9,9,Tiruchchendur,IN,8.4833,78.1167,79.45,80,69,6.29,broken clouds
11,11,Rawson,AR,-43.3002,-65.1023,68.92,45,1,26.28,clear sky
14,14,Campos Novos,BR,-27.4017,-51.2250,65.50,70,87,6.17,overcast clouds
17,17,Castro,BR,-24.7911,-50.0119,65.89,68,37,9.13,scattered clouds
24,24,Cabedelo,BR,-6.9811,-34.8339,79.47,82,18,11.79,few clouds
25,25,Kaabong,UG,3.4836,34.1492,65.12,72,22,12.21,few clouds
26,26,Bengkulu,ID,-3.8004,102.2655,75.27,89,100,8.12,overcast clouds
28,28,East London,ZA,-33.0153,27.9116,68.88,95,100,18.81,light rain


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,69.24,light rain,19.7297,-155.0900,
7,Bathsheba,BB,79.41,scattered clouds,13.2167,-59.5167,
9,Tiruchchendur,IN,79.45,broken clouds,8.4833,78.1167,
11,Rawson,AR,68.92,clear sky,-43.3002,-65.1023,
14,Campos Novos,BR,65.50,overcast clouds,-27.4017,-51.2250,
17,Castro,BR,65.89,scattered clouds,-24.7911,-50.0119,
24,Cabedelo,BR,79.47,few clouds,-6.9811,-34.8339,
25,Kaabong,UG,65.12,few clouds,3.4836,34.1492,
26,Bengkulu,ID,75.27,overcast clouds,-3.8004,102.2655,
28,East London,ZA,68.88,light rain,-33.0153,27.9116,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    #Add the lat and long to location key for params
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")
hotel_df.head(10)
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,69.24,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Bathsheba,BB,79.41,scattered clouds,13.2167,-59.5167,Atlantis Hotel
9,Tiruchchendur,IN,79.45,broken clouds,8.4833,78.1167,Hotel Udhayam International
11,Rawson,AR,68.92,clear sky,-43.3002,-65.1023,Hosteria Sampedro
14,Campos Novos,BR,65.50,overcast clouds,-27.4017,-51.2250,Hotel Corujão - Campos Novos
17,Castro,BR,65.89,scattered clouds,-24.7911,-50.0119,CHACARA BAILLY
24,Cabedelo,BR,79.47,few clouds,-6.9811,-34.8339,Hotel Pigalle
25,Kaabong,UG,65.12,few clouds,3.4836,34.1492,Rock Motel Ltd
26,Bengkulu,ID,75.27,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
28,East London,ZA,68.88,light rain,-33.0153,27.9116,Tu Casa


In [18]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [20]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))